# Project 1 – Charles – Metrics

In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

In [10]:
data = pd.read_csv("Cleaned_Data.csv", low_memory=False)
data.columns

Index(['name_x', 'distinct_id', 'time', 'ae_session_length', 'city', 'region',
       'mp_country_code', 'locationSetting', 'notificationSettings',
       'FirstTimeUploaded', 'Signup With', 'imageAdded', 'commentAdded',
       'commentLength', 'ImageUploaded', 'ae_total_app_session_length',
       'ae_total_app_sessions', 'timezone', 'last_seen', 'ios_app_version',
       'ios_version', 'ios_device_model', 'ios_lib_version',
       'ae_first_app_open_date', 'active_timespan', 'average_session_time',
       'country', 'US', 'Facebook', 'Foodie', 'Google', 'Unknown'],
      dtype='object')